## Import stuff

In [54]:
import tensorflow as tf
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from tensorflow.keras import layers
import shutil
import random
from PIL import Image
import numpy as np
import logging
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold
import torch 
import torch.jit
import torch.nn as nn 
import torchvision
from torchvision import transforms, datasets
import sklearn.metrics as metrics
logger = tf.get_logger()
logger.setLevel(logging.ERROR)
os.environ["CUDA_VISIBLE_DEVICES"]="0,1"

print(tf.__version__)

from tensorflow.python.client import device_lib
print("tensorflow doesn't detects GPU: ")
print(device_lib.list_local_devices() )
print(tf.test.gpu_device_name())
print(tf.config.list_physical_devices())

print("torch detects GPU: " , torch.cuda.is_available())


2.3.0
tensorflow doesn't detects GPU: 
[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 17265062595943054103
]

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]
torch detects GPU:  True


In [55]:


def get_score(df_true, df_submission):
    for i in range(df_true.shape[1]):
        print("score for col ",i,": " , metrics.roc_auc_score(df_true[:,i], df_submission[:,i]))


## Load training data from csv


In [56]:
features_df_orig = pd.read_csv("train_features_fixed.csv")
features_df_normalized = pd.read_csv("train_features_normalized.csv")
labels_df = pd.read_csv("train_labels.csv")


# load all columns except the first two
X_orig = np.array(features_df_orig.iloc[:, 2:].values,  dtype=float)
X_orig = X_orig.reshape(227940//12,35*12)
X_normalized = np.array(features_df_normalized.iloc[:, 2:].values,  dtype=float)
X_normalized = X_normalized.reshape(227940//12,35*12)


#use train to validation ratio of 0.8
X_orig_train = X_orig[:int(X_orig.shape[0]*0.8)]
X_orig_valid = X_orig[int(X_orig.shape[0]*0.8):]
X_normalized_train = X_normalized[:int(X_normalized.shape[0]*0.8)]
X_normalized_valid = X_normalized[int(X_normalized.shape[0]*0.8):]

Y1 = np.array(labels_df.iloc[:, 1:11].values,  dtype=float)
Y1_train = Y1[:int(Y1.shape[0]*0.8)]
Y1_valid = Y1[int(Y1.shape[0]*0.8):]

Y2 = np.array(labels_df.iloc[:, 11].values,  dtype=float)
Y2_train = Y2[:int(Y2.shape[0]*0.8)]
Y2_valid = Y2[int(Y2.shape[0]*0.8):]

Y3 = np.array(labels_df.iloc[:, 12:].values,  dtype=float)
Y3_train = Y3[:int(Y3.shape[0]*0.8)]
Y3_valid = Y3[int(Y3.shape[0]*0.8):]

print("X_orig shape: ", X_orig.shape)
print("X_normalized shape: ", X_normalized.shape)
print("Y1 shape: ", Y1.shape)
print("Y2 shape: ", Y2.shape)
print("Y3 shape: ", Y3.shape)



# print(inputs_dim)
# print(outputs_dim)

# X = tf.cast(X, tf.float32)
# Y = tf.cast(Y, tf.float32)

#print(X.shape)
# random.shuffle(image_ids)
# random.shuffle(diagnostics)

X_orig shape:  (18995, 420)
X_normalized shape:  (18995, 420)
Y1 shape:  (18995, 10)
Y2 shape:  (18995,)
Y3 shape:  (18995, 4)


## TASK 1: ORDERING OF MEDICAL TEST

In [63]:
inputs_dim = X_normalized.shape[1]
outputs_dim = Y1.shape[1]

model1 = tf.keras.Sequential([
  tf.keras.layers.Dense(units=100, input_shape=[inputs_dim], activation='sigmoid'),
  tf.keras.layers.Dense(units=25, activation='sigmoid'),
  tf.keras.layers.Dense(units=outputs_dim)
])

model1.compile(loss='mean_squared_error', optimizer=tf.keras.optimizers.Adam(0.1), metrics=['mean_squared_error'])

history = model1.fit(X_normalized_train, Y1_train, epochs=10) # , verbose=False
print("Finished training the model")

Epoch 1/10
475/475 [==============================] - 0s 873us/step - loss: 0.1523 - mean_squared_error: 0.1523
Epoch 2/10
475/475 [==============================] - 0s 830us/step - loss: 0.1440 - mean_squared_error: 0.1440
Epoch 3/10
475/475 [==============================] - 0s 700us/step - loss: 0.1444 - mean_squared_error: 0.1444
Epoch 4/10
475/475 [==============================] - 0s 692us/step - loss: 0.1450 - mean_squared_error: 0.1450
Epoch 5/10
475/475 [==============================] - 0s 679us/step - loss: 0.1447 - mean_squared_error: 0.1447
Epoch 6/10
475/475 [==============================] - 0s 676us/step - loss: 0.1457 - mean_squared_error: 0.1457
Epoch 7/10
475/475 [==============================] - 0s 688us/step - loss: 0.1467 - mean_squared_error: 0.1467
Epoch 8/10
475/475 [==============================] - 0s 691us/step - loss: 0.1454 - mean_squared_error: 0.1454
Epoch 9/10
475/475 [==============================] - 0s 697us/step - loss: 0.1468 - mean_squared_error:

## TASK 1: EVALUATION

In [64]:
pred = model1.predict(X_normalized_valid)
for i in range(Y1_valid.shape[1]):
    print("score for col ",i,": " , metrics.roc_auc_score(Y1_valid[:,i], pred[:,i]))



score for col  0 :  0.5069146988517818
score for col  1 :  0.5109629561746497
score for col  2 :  0.5
score for col  3 :  0.5
score for col  4 :  0.49412719886617706
score for col  5 :  0.5
score for col  6 :  0.5
score for col  7 :  0.5
score for col  8 :  0.5103490053144917
score for col  9 :  0.49727675400404475


## TASK 2: SEPSIS PREDICTION

In [59]:
inputs_dim = X_normalized.shape[1]
outputs_dim = 1

model2 = tf.keras.Sequential([
  tf.keras.layers.Dense(units=100, input_shape=[inputs_dim], activation='relu'),
  tf.keras.layers.Dense(units=25, activation='relu'),
  tf.keras.layers.Dense(units=outputs_dim)
])

model2.compile(loss='mean_squared_error', optimizer=tf.keras.optimizers.Adam(0.1), metrics=['mean_squared_error'])

history = model2.fit(X_normalized_train, Y2_train, epochs=5) # , verbose=False
print("Finished training the model")

Epoch 1/5
475/475 [==============================] - 0s 681us/step - loss: 19.5741 - mean_squared_error: 19.5741
Epoch 2/5
475/475 [==============================] - 0s 645us/step - loss: 0.0630 - mean_squared_error: 0.0630
Epoch 3/5
475/475 [==============================] - 0s 699us/step - loss: 0.0560 - mean_squared_error: 0.05600s - loss: 0.0570 - mean_squared_err
Epoch 4/5
475/475 [==============================] - 0s 702us/step - loss: 0.0550 - mean_squared_error: 0.0550
Epoch 5/5
475/475 [==============================] - 0s 750us/step - loss: 0.0550 - mean_squared_error: 0.0550
Finished training the model


## TASK 2: EVALUATION

In [60]:
pred = model2.predict(X_normalized_valid)
print("score : " , metrics.roc_auc_score(Y1_valid, pred))

score :  0.5


## TASK 3: KEYS VITALS SIGNS PREDICTION

In [61]:
inputs_dim = X_orig.shape[1]
outputs_dim = Y3.shape[1]

model3 = tf.keras.Sequential([
  tf.keras.layers.Dense(units=100, input_shape=[inputs_dim], activation='relu'),
  tf.keras.layers.Dense(units=25, activation='relu'),
  tf.keras.layers.Dense(units=outputs_dim)
])

model3.compile(loss='mean_squared_error', optimizer=tf.keras.optimizers.Adam(0.1), metrics=['mean_squared_error'])

history = model3.fit(X_orig_train, Y3_train, epochs=5) # , verbose=False
print("Finished training the model")

Epoch 1/5
475/475 [==============================] - 0s 657us/step - loss: 217983.8594 - mean_squared_error: 217983.8594
Epoch 2/5
475/475 [==============================] - 0s 661us/step - loss: 126.5711 - mean_squared_error: 126.5711
Epoch 3/5
475/475 [==============================] - 0s 664us/step - loss: 102.7951 - mean_squared_error: 102.7951
Epoch 4/5
475/475 [==============================] - 0s 676us/step - loss: 132.3274 - mean_squared_error: 132.3274
Epoch 5/5
475/475 [==============================] - 0s 655us/step - loss: 106.8408 - mean_squared_error: 106.8408
Finished training the model


## TASK 3: EVALUATION

In [62]:
pred = model3.predict(X_orig_valid)
for i in range(Y3_valid.shape[1]):
    print("score for col ",i,": " , 0.5 + 0.5 * np.maximum(0, metrics.r2_score(Y3_valid[i], pred[i])))



score for col  0 :  0.9781298142088239
score for col  1 :  0.9921901556178777
score for col  2 :  0.9255223387803362
score for col  3 :  0.8847010520610992


## Writing results